In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1337)

In [2]:
data = pd.read_excel('F:/DS/UBPhilanthropy/Management_final_1.xlsx')

In [3]:
data.head()

,ID_NUMBER,FACULTY_STAFF_IND,MARITAL_STATUS,GENDER,AGE_BRACKET,STATE_CODE,ZIPCODE_NEW,ZIPCODE,STUDENT_ACTIVITY_PARTICIPANT,DEGREE_CODE1,...,NO_EVENTS_2017_NEW,NO_EVENTS_2016_NEW,NO_EVENTS_2015_NEW,NO_EVENTS_2014_NEW,NO_EVENTS_2013_NEW,NO_EVENTS_2012_NEW,NO_EVENTS_2011_NEW,NO_EVENTS_2010_NEW,NO_EVENTS_2009_NEW,NO_EVENTS_2008_NEW
0,109248,N,S,M,90,NY,10031,10031,N,BS,...,0,0,0,0,0,0,0,0,0,1
1,52671,N,S,M,60,NY,11111,13027,N,BS,...,0,0,0,0,0,0,0,0,0,0
2,159329,N,M,M,45,NY,11111,14043,N,BS,...,0,0,0,0,0,0,0,0,0,0
3,52861,N,,M,55,NY,10031,10031,N,BS,...,0,0,0,0,0,0,0,0,0,0
4,37191,N,M,M,80,NaN,11111,NaN,N,BS,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.shape

(34338, 54)

In [5]:
data.isnull().sum()

ID_NUMBER                           0
FACULTY_STAFF_IND                   0
MARITAL_STATUS                      0
GENDER                              0
AGE_BRACKET                         0
STATE_CODE                        291
ZIPCODE_NEW                         0
ZIPCODE                           292
STUDENT_ACTIVITY_PARTICIPANT        0
DEGREE_CODE1                        0
DEGREE_YEAR1                        0
SCHOOL1                             0
DEGREE_CODE2                        0
DEGREE_YEAR2                        0
SCHOOL2                         28807
DEGREE_CODE3                        0
DEGREE_YEAR3                        0
SCHOOL3                         33772
DEGREE_CODE4                        0
DEGREE_YEAR4                        0
SCHOOL4                         34300
DONOR_CATEGORY                      0
LIFETIME_RECOGNITION_AMT_NEW        0
LIFETIME_RECOGNITION_AMT        23303
NO_YRS_GIVING                   23083
GIFT_AMOUNT_2018_NEW                0
GIFT_AMOUNT_

In [6]:
threshold = 5
cols = data.columns.tolist()
drop_cols = []
for col in cols:
    if data[col].isnull().sum() > len(data)*threshold/100:
        drop_cols.append(col)
        
data.drop(columns=drop_cols, inplace=True)

In [7]:
data.shape

(34338, 43)

In [8]:
print('Total numerical valued columns: ', len(data.select_dtypes(include=np.number).columns), '\n')
for col in data.select_dtypes(include=np.number).columns:
    print(col, data[col].isnull().sum())

Total numerical valued columns:  28 

ID_NUMBER 0
AGE_BRACKET 0
ZIPCODE_NEW 0
DEGREE_YEAR1 0
LIFETIME_RECOGNITION_AMT_NEW 0
GIFT_AMOUNT_2018_NEW 0
GIFT_AMOUNT_2017_NEW 0
GIFT_AMOUNT_2016_NEW 0
GIFT_AMOUNT_2015_NEW 0
GIFT_AMOUNT_2014_NEW 0
GIFT_AMOUNT_2013_NEW 0
GIFT_AMOUNT_2012_NEW 0
GIFT_AMOUNT_2011_NEW 0
GIFT_AMOUNT_2010_NEW 0
GIFT_AMOUNT_2009_NEW 0
GIFT_AMOUNT_2008_NEW 0
ENGAGEMENT_SCORE 0
NO_EVENTS_2018_NEW 0
NO_EVENTS_2017_NEW 0
NO_EVENTS_2016_NEW 0
NO_EVENTS_2015_NEW 0
NO_EVENTS_2014_NEW 0
NO_EVENTS_2013_NEW 0
NO_EVENTS_2012_NEW 0
NO_EVENTS_2011_NEW 0
NO_EVENTS_2010_NEW 0
NO_EVENTS_2009_NEW 0
NO_EVENTS_2008_NEW 0


In [9]:
print('Total categorical valued columns: ', len(data.select_dtypes(include=np.object).columns), '\n')
for col in data.select_dtypes(include=np.object).columns:
    print(col, data[col].isnull().sum())

Total categorical valued columns:  15 

FACULTY_STAFF_IND 0
MARITAL_STATUS 0
GENDER 0
STATE_CODE 291
ZIPCODE 292
STUDENT_ACTIVITY_PARTICIPANT 0
DEGREE_CODE1 0
SCHOOL1 0
DEGREE_CODE2 0
DEGREE_YEAR2 0
DEGREE_CODE3 0
DEGREE_YEAR3 0
DEGREE_CODE4 0
DEGREE_YEAR4 0
DONOR_CATEGORY 0


In [10]:
data['DEGREE_YEAR1'] = data['DEGREE_YEAR1'].astype('object')
data['AGE_BRACKET'] = data['AGE_BRACKET'].astype('object')

In [11]:
# #Taking the top 9 correlated features from above and dropping the rest of the features
# # drop_cols = data.corr()['GIFT_AMOUNT_2018_NEW'].sort_values(ascending=False).tail(17).index.tolist()

# #Retaining ID number for now and dropping it after splitting into train and test sets.
# drop_cols.remove('ID_NUMBER')
# data.drop(columns=drop_cols, inplace=True)

In [12]:
data.shape

(34338, 43)

In [13]:
cat_cols = data.select_dtypes(include=np.object).columns.tolist()

num_cols = data.select_dtypes(include=np.number).columns.tolist()

In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import Callback, EarlyStopping
from keras.optimizers import RMSprop, Adadelta, Adagrad

Using TensorFlow backend.


In [15]:
data.columns

Index(['ID_NUMBER', 'FACULTY_STAFF_IND', 'MARITAL_STATUS', 'GENDER',
       'AGE_BRACKET', 'STATE_CODE', 'ZIPCODE_NEW', 'ZIPCODE',
       'STUDENT_ACTIVITY_PARTICIPANT', 'DEGREE_CODE1', 'DEGREE_YEAR1',
       'SCHOOL1', 'DEGREE_CODE2', 'DEGREE_YEAR2', 'DEGREE_CODE3',
       'DEGREE_YEAR3', 'DEGREE_CODE4', 'DEGREE_YEAR4', 'DONOR_CATEGORY',
       'LIFETIME_RECOGNITION_AMT_NEW', 'GIFT_AMOUNT_2018_NEW',
       'GIFT_AMOUNT_2017_NEW', 'GIFT_AMOUNT_2016_NEW', 'GIFT_AMOUNT_2015_NEW',
       'GIFT_AMOUNT_2014_NEW', 'GIFT_AMOUNT_2013_NEW', 'GIFT_AMOUNT_2012_NEW',
       'GIFT_AMOUNT_2011_NEW', 'GIFT_AMOUNT_2010_NEW', 'GIFT_AMOUNT_2009_NEW',
       'GIFT_AMOUNT_2008_NEW', 'ENGAGEMENT_SCORE', 'NO_EVENTS_2018_NEW',
       'NO_EVENTS_2017_NEW', 'NO_EVENTS_2016_NEW', 'NO_EVENTS_2015_NEW',
       'NO_EVENTS_2014_NEW', 'NO_EVENTS_2013_NEW', 'NO_EVENTS_2012_NEW',
       'NO_EVENTS_2011_NEW', 'NO_EVENTS_2010_NEW', 'NO_EVENTS_2009_NEW',
       'NO_EVENTS_2008_NEW'],
      dtype='object')

In [16]:
def checkMAE(actual, pred, n):
    count = 0
    for i in range(len(actual)):
        if abs(actual[i] - pred[i]) > n:
            count +=1
    
    return count

mae_values = [10, 20, 50, 75, 100]

In [17]:
df = data.copy(deep=True)
# df = df[num_cols]
df = pd.get_dummies(df)
X = df.drop(columns='GIFT_AMOUNT_2018_NEW')
y = df['GIFT_AMOUNT_2018_NEW']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Saving id number for future use
train_id_number = X_train['ID_NUMBER']
test_id_number = X_test['ID_NUMBER']
X_train.drop(columns=['ID_NUMBER'], inplace=True)
X_test.drop(columns=['ID_NUMBER'], inplace=True)

In [18]:
df.shape

(34338, 8445)

In [19]:
#Keras Neural Network
model = Sequential()
#Input layer
model.add(Dense(128, activation='relu', input_shape=[len(X_train.keys())]))
#Hidden layers
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
#Output layer
model.add(Dense(1, activation='relu'))

optimizer = Adadelta(lr=0.001)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error', 'mean_absolute_error'])

W0813 10:44:56.283142 12044 deprecation_wrapper.py:119] From C:\Users\Abhishek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 10:44:56.695331 12044 deprecation_wrapper.py:119] From C:\Users\Abhishek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 10:44:56.761173 12044 deprecation_wrapper.py:119] From C:\Users\Abhishek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 10:44:56.901787 12044 deprecation_wrapper.py:119] From C:\Users\Abhishek\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [20]:
epoch = 10
history = model.fit(X_train, y_train, epochs=epoch, verbose=0)

W0813 10:44:58.250748 12044 deprecation_wrapper.py:119] From C:\Users\Abhishek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0813 10:44:58.254748 12044 deprecation_wrapper.py:119] From C:\Users\Abhishek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



In [21]:
loss, mse, mae = model.evaluate(X_train, y_train, verbose=0)
print('\t\t\t\tKeras Dense Neural Network with all numerical and categorical features\n')
print('Training data accuracy')
print('Length of Training Data: ', len(X_train))
print('MSE: ', mse)
print('MAE: ', mae)
keras_train_pred = model.predict(X_train)
keras_train_pred = [pred for sublist in keras_train_pred.tolist() for pred in sublist]
for i in mae_values:
    print('Number of predicted values with  absolute error > ', i, ': ', checkMAE(y_train.values.tolist(), keras_train_pred, i),
     '(', '%.2f' % (checkMAE(y_train.values.tolist(), keras_train_pred, i) / len(y_train) * 100), '%)')
print('\n')
loss, mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('Test data accuracy')
print('Length of Testing Data: ', len(X_test))
print('MSE: ', mse)
print('MAE: ', mae)
keras_test_pred = model.predict(X_test)
keras_test_pred = [pred for sublist in keras_test_pred.tolist() for pred in sublist]
for i in mae_values:
    print('Number of predicted values with  absolute error > ', i, ': ', checkMAE(y_test.values.tolist(), keras_test_pred, i),
     '(', '%.2f' % (checkMAE(y_test.values.tolist(), keras_test_pred, i) / len(y_test) * 100), '%)')

				Keras Dense Neural Network with all numerical and categorical features

Training data accuracy
Length of Training Data:  27470
MSE:  557262.2243901873
MAE:  24.901181662468897
Number of predicted values with  absolute error >  10 :  1248 ( 4.54 %)
Number of predicted values with  absolute error >  20 :  1207 ( 4.39 %)
Number of predicted values with  absolute error >  50 :  797 ( 2.90 %)
Number of predicted values with  absolute error >  75 :  748 ( 2.72 %)
Number of predicted values with  absolute error >  100 :  503 ( 1.83 %)


Test data accuracy
Length of Testing Data:  6868
MSE:  175615.60866115172
MAE:  25.882762101894905
Number of predicted values with  absolute error >  10 :  308 ( 4.48 %)
Number of predicted values with  absolute error >  20 :  296 ( 4.31 %)
Number of predicted values with  absolute error >  50 :  212 ( 3.09 %)
Number of predicted values with  absolute error >  75 :  200 ( 2.91 %)
Number of predicted values with  absolute error >  100 :  138 ( 2.01 %)


In [22]:
keras_train_df = pd.DataFrame(data=list(zip(train_id_number, y_train, keras_train_pred)), 
                              columns=['ID_NUmber', 'GIFT_AMOUNT_2018_NEW', 'PREDICTED_GIFT_AMOUNT_2018'])
keras_train_df.to_csv('F:/DS/UBPhilanthropy/34kKerasADTrainPred.csv', index=False)

keras_test_df = pd.DataFrame(data = list(zip(test_id_number, y_test, keras_test_pred)), 
                              columns=['ID_NUmber', 'GIFT_AMOUNT_2018_NEW', 'PREDICTED_GIFT_AMOUNT_2018'])
keras_test_df.to_csv('F:/DS/UBPhilanthropy/34kKerasADTestPred.csv', index=False)

In [23]:
rf = RandomForestRegressor()

In [24]:
# param_grid = {'n_estimators': [50, 100, 150, 200 ,250],
#               'max_depth': [3, 4, 5, 6, 7]
#              }
# grid = GridSearchCV(estimator=rf, param_grid=param_grid, refit=True)
# grid.fit(X_train, y_train)
# grid.best_params_

In [25]:
rf.n_estimators = 100
rf.max_depth = 7
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [26]:
rf_train_pred = rf.predict(X_train)
print('\t\t\t\tRandom Forest Regressor with all features\n')
print('Training data accuracy')
print('Length of Training Data: ', len(X_train))
print('MSE: ', mean_squared_error(y_train, rf_train_pred))
print('MAE: ', mean_absolute_error(y_train, rf_train_pred))
for i in mae_values:
    print('Number of predicted values with  absolute error > ', i, ': ', checkMAE(y_train.values.tolist(), rf_train_pred.tolist(), i),
     '(', '%.2f' % (checkMAE(y_train.values.tolist(), rf_train_pred.tolist(), i) / len(y_train) * 100), '%)')
#print('r^2: ', r2_score(y_train, rf_train_pred))

print('\n')
rf_test_pred = rf.predict(X_test)
print('Testing data accuracy')
print('Length of Testing Data: ', len(X_test))
print('MSE: ', mean_squared_error(y_test, rf_test_pred))
print('MAE: ', mean_absolute_error(y_test, rf_test_pred))
for i in mae_values:
    print('Number of predicted values with absolute error > ', i, ': ', checkMAE(y_test.values.tolist(), rf_test_pred.tolist(), i),
     '(', '%.2f' % (checkMAE(y_test.values.tolist(), rf_test_pred.tolist(), i) / len(y_test) * 100), '%)')
#print('r^2: ', r2_score(y_test, rf_test_pred))

				Random Forest Regressor with all features

Training data accuracy
Length of Training Data:  27470
MSE:  100323.3218013063
MAE:  10.618444489301735
Number of predicted values with  absolute error >  10 :  1315 ( 4.79 %)
Number of predicted values with  absolute error >  20 :  1095 ( 3.99 %)
Number of predicted values with  absolute error >  50 :  669 ( 2.44 %)
Number of predicted values with  absolute error >  75 :  451 ( 1.64 %)
Number of predicted values with  absolute error >  100 :  362 ( 1.32 %)


Testing data accuracy
Length of Testing Data:  6868
MSE:  78766.46635040545
MAE:  19.19001264324287
Number of predicted values with absolute error >  10 :  330 ( 4.80 %)
Number of predicted values with absolute error >  20 :  282 ( 4.11 %)
Number of predicted values with absolute error >  50 :  179 ( 2.61 %)
Number of predicted values with absolute error >  75 :  128 ( 1.86 %)
Number of predicted values with absolute error >  100 :  101 ( 1.47 %)


In [27]:
X_train.columns

Index(['ZIPCODE_NEW', 'LIFETIME_RECOGNITION_AMT_NEW', 'GIFT_AMOUNT_2017_NEW',
       'GIFT_AMOUNT_2016_NEW', 'GIFT_AMOUNT_2015_NEW', 'GIFT_AMOUNT_2014_NEW',
       'GIFT_AMOUNT_2013_NEW', 'GIFT_AMOUNT_2012_NEW', 'GIFT_AMOUNT_2011_NEW',
       'GIFT_AMOUNT_2010_NEW',
       ...
       'DEGREE_YEAR4_2010', 'DEGREE_YEAR4_2011', 'DEGREE_YEAR4_2012',
       'DEGREE_YEAR4_2013', 'DEGREE_YEAR4_ ', 'DONOR_CATEGORY_1st Time    ',
       'DONOR_CATEGORY_Lapsed      ', 'DONOR_CATEGORY_Loyal       ',
       'DONOR_CATEGORY_Non Donor   ', 'DONOR_CATEGORY_Reacquired  '],
      dtype='object', length=8443)

In [28]:
rf.feature_importances_

array([0.00634506, 0.24145561, 0.0434477 , ..., 0.01945798, 0.        ,
       0.03173017])

In [29]:
pd.DataFrame(data=list(zip(X_train.columns.tolist(), rf.feature_importances_.tolist())),
             columns=['Feature', 'Importance']
            ).sort_values(by='Importance').tail(10)

,Feature,Importance
14,NO_EVENTS_2017_NEW,0.023930
17,NO_EVENTS_2014_NEW,0.024464
8442,DONOR_CATEGORY_Reacquired,0.031730
3,GIFT_AMOUNT_2016_NEW,0.033148
2,GIFT_AMOUNT_2017_NEW,0.043448
2152,ZIPCODE_14424,0.077759
8438,DONOR_CATEGORY_1st Time,0.087092
4,GIFT_AMOUNT_2015_NEW,0.092424
8151,DEGREE_YEAR1_1979,0.109371
1,LIFETIME_RECOGNITION_AMT_NEW,0.241456


In [30]:
rf_train_df = pd.DataFrame(data=list(zip(train_id_number, y_train, rf_train_pred)), 
                              columns=['ID_NUmber', 'GIFT_AMOUNT_2018_NEW', 'PREDICTED_GIFT_AMOUNT_2018'])
rf_train_df.to_csv('F:/DS/UBPhilanthropy/34kRFTrainPred.csv', index=False)

rf_test_df = pd.DataFrame(data = list(zip(test_id_number, y_test, rf_test_pred)), 
                              columns=['ID_NUmber', 'GIFT_AMOUNT_2018_NEW', 'PREDICTED_GIFT_AMOUNT_2018'])
rf_test_df.to_csv('F:/DS/UBPhilanthropy/34kRFTestPred.csv', index=False)

In [31]:
sel_cols = ['NO_EVENTS_2017_NEW','NO_EVENTS_2016_NEW','NO_EVENTS_2015_NEW','NO_EVENTS_2014_NEW','GIFT_AMOUNT_2018_NEW',
            'GIFT_AMOUNT_2017_NEW','GIFT_AMOUNT_2016_NEW','GIFT_AMOUNT_2015_NEW','LIFETIME_RECOGNITION_AMT_NEW',
            'ZIPCODE_NEW']
sel_cols.append('ID_NUMBER')

In [32]:
df = data.copy(deep=True)
df = df[sel_cols]
df = pd.get_dummies(df)
X = df.drop(columns='GIFT_AMOUNT_2018_NEW')
y = df['GIFT_AMOUNT_2018_NEW']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Saving id number for future use
train_id_number = X_train['ID_NUMBER']
test_id_number = X_test['ID_NUMBER']
X_train.drop(columns=['ID_NUMBER'], inplace=True)
X_test.drop(columns=['ID_NUMBER'], inplace=True)

In [33]:
X_train.keys()

Index(['NO_EVENTS_2017_NEW', 'NO_EVENTS_2016_NEW', 'NO_EVENTS_2015_NEW',
       'NO_EVENTS_2014_NEW', 'GIFT_AMOUNT_2017_NEW', 'GIFT_AMOUNT_2016_NEW',
       'GIFT_AMOUNT_2015_NEW', 'LIFETIME_RECOGNITION_AMT_NEW', 'ZIPCODE_NEW'],
      dtype='object')

In [34]:
#Keras Neural Network
model = Sequential()
#Input layer
model.add(Dense(128, activation='relu', input_shape=[len(X_train.keys())]))
#Hidden layers
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))

#Output layers
model.add(Dense(1, activation='relu'))

optimizer = Adadelta(lr=0.001)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error', 'mean_absolute_error'])

In [35]:
epoch = 10
history = model.fit(X_train, y_train, epochs=epoch, verbose=0)

In [36]:
loss, mse, mae = model.evaluate(X_train, y_train, verbose=0)
print('\t\t\t\tKeras Dense Neural Network with selected numerical features\n')
print('Training data accuracy')
print('Length of Training Data: ', len(X_train))
print('MSE: ', mse)
print('MAE: ', mae)
keras_train_pred = model.predict(X_train)
keras_train_pred = [pred for sublist in keras_train_pred.tolist() for pred in sublist]
for i in mae_values:
    print('Number of predicted values with  absolute error > ', i, ': ', checkMAE(y_train.values.tolist(), keras_train_pred, i),
     '(', '%.2f' % (checkMAE(y_train.values.tolist(), keras_train_pred, i) / len(y_train) * 100), '%)')
print('\n')
loss, mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('Test data accuracy')
print('Length of Testing Data: ', len(X_test))
print('MSE: ', mse)
print('MAE: ', mae)
keras_test_pred = model.predict(X_test)
keras_test_pred = [pred for sublist in keras_test_pred.tolist() for pred in sublist]
for i in mae_values:
    print('Number of predicted values with  absolute error > ', i, ': ', checkMAE(y_test.values.tolist(), keras_test_pred, i),
     '(', '%.2f' % (checkMAE(y_test.values.tolist(), keras_test_pred, i) / len(y_test) * 100), '%)')

				Keras Dense Neural Network with selected numerical features

Training data accuracy
Length of Training Data:  27470
MSE:  557262.2243901873
MAE:  24.901181662468897
Number of predicted values with  absolute error >  10 :  1248 ( 4.54 %)
Number of predicted values with  absolute error >  20 :  1207 ( 4.39 %)
Number of predicted values with  absolute error >  50 :  797 ( 2.90 %)
Number of predicted values with  absolute error >  75 :  748 ( 2.72 %)
Number of predicted values with  absolute error >  100 :  503 ( 1.83 %)


Test data accuracy
Length of Testing Data:  6868
MSE:  175615.60866115172
MAE:  25.882762101894905
Number of predicted values with  absolute error >  10 :  308 ( 4.48 %)
Number of predicted values with  absolute error >  20 :  296 ( 4.31 %)
Number of predicted values with  absolute error >  50 :  212 ( 3.09 %)
Number of predicted values with  absolute error >  75 :  200 ( 2.91 %)
Number of predicted values with  absolute error >  100 :  138 ( 2.01 %)


In [37]:
rf = RandomForestRegressor()
rf.n_estimators = 5
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [38]:
rf_train_pred = rf.predict(X_train)
print('\t\t\t\tRandom Forest Regressor with selected numerical features\n')
print('Training data accuracy')
print('Length of Training Data: ', len(X_train))
print('MSE: ', mean_squared_error(y_train, rf_train_pred))
print('MAE: ', mean_absolute_error(y_train, rf_train_pred))
for i in mae_values:
    print('Number of predicted values with  absolute error > ', i, ': ', checkMAE(y_train.values.tolist(), rf_train_pred.tolist(), i),
     '(', '%.2f' % (checkMAE(y_train.values.tolist(), rf_train_pred.tolist(), i) / len(y_train) * 100), '%)')
#print('r^2: ', r2_score(y_train, rf_train_pred))

print('\n')
rf_test_pred = rf.predict(X_test)
print('Testing data accuracy')
print('Length of Testing Data: ', len(X_test))
print('MSE: ', mean_squared_error(y_test, rf_test_pred))
print('MAE: ', mean_absolute_error(y_test, rf_test_pred))
for i in mae_values:
    print('Number of predicted values with absolute error > ', i, ': ', checkMAE(y_test.values.tolist(), rf_test_pred.tolist(), i),
     '(', '%.2f' % (checkMAE(y_test.values.tolist(), rf_test_pred.tolist(), i) / len(y_test) * 100), '%)')
#print('r^2: ', r2_score(y_test, rf_test_pred))

				Random Forest Regressor with selected numerical features

Training data accuracy
Length of Training Data:  27470
MSE:  91120.12568260545
MAE:  11.486534420318065
Number of predicted values with  absolute error >  10 :  1522 ( 5.54 %)
Number of predicted values with  absolute error >  20 :  1098 ( 4.00 %)
Number of predicted values with  absolute error >  50 :  585 ( 2.13 %)
Number of predicted values with  absolute error >  75 :  452 ( 1.65 %)
Number of predicted values with  absolute error >  100 :  323 ( 1.18 %)


Testing data accuracy
Length of Testing Data:  6868
MSE:  587995.8895637055
MAE:  39.052544698473376
Number of predicted values with absolute error >  10 :  540 ( 7.86 %)
Number of predicted values with absolute error >  20 :  446 ( 6.49 %)
Number of predicted values with absolute error >  50 :  295 ( 4.30 %)
Number of predicted values with absolute error >  75 :  236 ( 3.44 %)
Number of predicted values with absolute error >  100 :  177 ( 2.58 %)


In [39]:
data['DONOR_CATEGORY'].unique()

array(['Lapsed      ', 'Reacquired  ', 'Non Donor   ', 'Loyal       ',
       '1st Time    '], dtype=object)

In [40]:
data['DONOR_CATEGORY'] = data['DONOR_CATEGORY'].apply(lambda x: 1 if x == 'Loyal       ' else 0)

In [41]:
data['GIFT_AMOUNT_2018_NEW'] = data['GIFT_AMOUNT_2018_NEW'].apply(lambda x: 1 if x>0 else 0)

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [43]:
dtc = DecisionTreeClassifier()
log = LogisticRegression()

In [44]:
df = data.copy(deep=True)
# df = df[num_cols]
df = pd.get_dummies(df)
X = df.drop(columns=['GIFT_AMOUNT_2018_NEW'])
y = df['GIFT_AMOUNT_2018_NEW']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Saving id number for future use
train_id_number = X_train['ID_NUMBER']
test_id_number = X_test['ID_NUMBER']
X_train.drop(columns=['ID_NUMBER'], inplace=True)
X_test.drop(columns=['ID_NUMBER'], inplace=True)

In [45]:
log.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [46]:
pred = log.predict(X_train)
test_pred = log.predict(X_test)
print(accuracy_score(y_train, pred))
print(confusion_matrix(y_train, pred))
print('\n')
print(accuracy_score(y_test, test_pred))
print(confusion_matrix(y_test, test_pred))


0.9549690571532581
[[26147    33]
 [ 1204    86]]


0.9551543389633081
[[6539    8]
 [ 300   21]]


In [47]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [48]:
pred = dtc.predict(X_train)
test_pred = dtc.predict(X_test)


print(accuracy_score(y_train, pred))
print(confusion_matrix(y_train, pred))
print('\n')
print(accuracy_score(y_test, test_pred))
print(confusion_matrix(y_test, test_pred))

1.0
[[26180     0]
 [    0  1290]]


0.9876237623762376
[[6508   39]
 [  46  275]]


In [49]:
test_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)